# ARIMA (SARIMA)

The script aims to forecast the values of a time series using an ARIMA (or SARIMA) model. The series in question, which can be found on this [link](https://www3.bcb.gov.br/sgspub/consultarvalores/telaCvsSelecionarSeries.paint) represents the Retail Sales Volume Index in the state of São Paulo, Brazil, with monthly data from 2000 to 2022.

The model will be trained using the data up to December 2020 and then be capable of making predictions for the years 2021 and 2022.

The chosen evaluation metric is MAPE (Mean Absolute Percentage Error), which is commonly used to evaluate forecasts in time series. MAPE is calculated as the average of the absolute percentage differences between actual and predicted values relative to actual values.

The formula for MAPE is given by:

$MAPE = \large\frac{1}{n}\small * ∑\large|\large\frac{(Yi - Pi)}{Yi}|\small * 100$

Where:

- n is the number of observations
- $Y_i$ is the actual value of observation i
- $P_i$ is the predicted value of observation i

## Implementation

- Check the dataset for patterns of trend and seasonality.


- Test the stationarity of the data using the Dickey-Fuller test.


- Perform differencing on the series.


- Create and train the model.


- Obtain the forecasts.


- Perform reverse differencing.


- Analyze the obtained results (outputs).

***

### Checking Dataset

In [8]:
# Libraries needed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
from pmdarima.arima.utils import ndiffs
import statsmodels.api as sm
import datetime as dt
import warnings

warnings.filterwarnings("ignore")
sns.set()

In [9]:
# Checking the data
df_retail = pd.read_csv(f'Data/retailsp.csv', index_col=0)
df_retail.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/retailsp.csv'

In [ ]:
# Transforming the Date column
df_retail['Date'] = pd.to_datetime(df_retail['Date'])

In [ ]:
# Plotting the Retail Value over the Date
plt.figure(figsize=(12,4))
plt.plot(df_retail['Date'], df_retail['value'])
plt.xlabel('Date')
plt.ylabel('Retail value')
plt.title('Retail value over the Date')
plt.axvline(dt.datetime(2021, 1, 1), color='red', linestyle='dotted')
plt.show();

The dotted line indicates the point at which the forecasts will begin to be made.

By looking at the plot, it is clearly evident that there is a seasonal pattern in the data (which would indicate that the SARIMA model should be used instead of ARIMA). However, it might be interesting to perform a double-check by decomposing the series.

In [ ]:
# Decomposing
decomposed_ts = sm.tsa.seasonal_decompose(df_retail['value'], period=12)

In [ ]:
# Plotting the Seasonality
plt.figure(figsize=(12,3))
plt.plot(df_retail['Date'], decomposed_ts.seasonal)
plt.xlabel('Date')
plt.ylabel('Seasonality')
plt.axvline(dt.datetime(2021, 1, 1), color='red', linestyle='dotted')
plt.show();

Decomposing the time series makes it even clearer that there is a seasonal component influencing the series on an annual basis, and this information should be considered in the model construction.

### Verifying stationarity

A stationary time series is one in which the statistical properties, such as mean, variance, and autocorrelation, do not change over time. In other words, the series maintains a consistent pattern and is not affected by trends, seasonality, or abrupt changes. The ARIMA model can only be applied to stationary series.

One way to test the stationarity of a series is by using the Dickey-Fuller test. This test helps determine whether a series is stationary by examining the presence of unit roots. If the test results in a p-value below a certain significance level (e.g., 0.05), it suggests that the series is stationary. Conversely, a p-value above the significance level indicates that the series is non-stationary and requires further treatment, such as differencing, to achieve stationarity.


In [ ]:
# Dickey Fuller Test
adf_result = adfuller(df_retail['value'], maxlag=1)
adf_result

The second value in the print is the p-value, which, being greater than 0.05, does not reject the null hypothesis, indicating that the series, under a 5% significance level, is stationary.

### Differencing the Series 

The series is not stationary; therefore, some form of treatment is necessary for applying the model.

The most commonly used treatment to remove non-stationary trends and patterns is differencing, which makes the series more suitable for modeling using the AR (autoregressive) and MA (moving average) components of ARIMA.

By applying differencing, it is possible to reduce the series' dependence on time and make it stationary in terms of mean and variance. This facilitates forecasting and modeling using ARIMA.

In [ ]:
# Determining the number of differentiations required
diffs = ndiffs(df_retail['value'], test='adf')
diffs

The series requires 1 differentiation, which means that it is necessary to subtract the values of the series over a time interval to make it stationary.

In [ ]:
# Differencing
dif_df_retail = np.diff(df_retail['value'])

In [ ]:
# Checking the values after the differencing process
print(dif_df_retail[0:5])
print(df_retail['value'].loc[1:5].values -  df_retail['value'].loc[0:4].values)

As we can observe, the obtained values correspond to the subtraction of an original value by its previous value.

### Create and train the model

It will be necessary to split the differenced series into training and testing sets. This is done to create and adjust the model and later make predictions that can be compared with the original values.

In [ ]:
# Getting the last value index
idx_split = df_retail.loc[df_retail['Date']==dt.datetime(2020,12,1)].index[0]

In [ ]:
# Splitting the train and test values
train_values = dif_df_retail[0:idx_split]
test_values = dif_df_retail[idx_split:]

The model will be created using the auto_arima function, which performs an automatic search to determine the best parameters for the ARIMA model based on the training data. The 'trace' parameter is set to True to display information about the model.

The seasonal parameter, previously mentioned, should be added to the function, and 'm=12' sets the seasonal periodicity as 12, indicating an annual seasonality.

In [ ]:
# Creating the model
arima_model = auto_arima(train_values, trace=True, seazonal=True, m=12)

The ARIMA(1,0,2)(0,1,1)[12] model was selected as the best model based on the automatic search performed. This model indicates that it has an autoregressive (AR) component of order 1, a moving average (MA) component of order 2, and a seasonal differencing component of order 1 with a period of 12 (indicating annual seasonality). These parameters were determined to be the most suitable for fitting the training data based on the Akaike Information Criterion (AIC).

After the selection, the model will be fitted to the training data using the determined parameters.

In [ ]:
# Fit
arima_model.fit(train_values)

In [ ]:
# Model Outputs
arima_model.summary()

### Making predictions

The now fitted model will make predictions corresponding to the size of the test data.

In [ ]:
# Predictions
y_pred = arima_model.predict(n_periods=len(test_values))

In [ ]:
y_pred

In [ ]:
test_values

Although the model's predictions may seem consistent with the test data, it is important to remember that these data are differenced and, therefore, the predictions do not correspond to the actual values of the dataset.

The usefulness of the model will need to be assessed after the reverse differencing procedure, where the predicted values can be compared with the actual values.

### Reverse differencing

The way to obtain reverse differentiation is by taking the last value of the training series and adding the values obtained from the predictions.

In [ ]:
# Getting the train database last value
train_last_value = df_retail['value'].loc[idx_split]
train_last_value

In [ ]:
# Getting the reversed predictions
r_pred = np.r_[train_last_value, y_pred].cumsum()[1:]

### Analyzing the results

Once the predicted values have undergone the reverse differentiation process, their values can be compared with those of the training dataset.

The Mean Absolute Percentage Error (MAPE) will be the metric used to evaluate the performance of the model.

In [ ]:
# Predicted and real values
df_test = df_retail.loc[idx_split+1:]
df_test['predicted_value'] = r_pred
df_test[['Date', 'value', 'predicted_value']]

In [ ]:
# Getting MAPE
print(f"MAPE: {round((np.mean(abs((df_test['value'] - df_test['predicted_value'])/df_test['value']))),4) * 100}% ")

A MAPE of 5.91% indicates that, on average, the model's predictions have a mean absolute percentage error of 5.91% compared to the actual values in the training dataset. A lower MAPE value indicates better model performance, as it indicates that the predictions are closer to the actual values. Therefore, a MAPE of 5.91% is considered a positive result, indicating good accuracy in the predictions.

In [ ]:
# Plotting the Retail Value and predictions after 2020
plt.figure(figsize=(12,4))
plt.plot(df_retail['Date'].loc[df_retail['Date']>dt.datetime(2020, 1, 1)], df_retail['value'].loc[df_retail['Date']>dt.datetime(2020, 1, 1)])
plt.plot(df_test['Date'], df_test['predicted_value'])
plt.xlabel('Date')
plt.title('Retail Value and Predictions')
plt.axvline(dt.datetime(2021, 1, 1), color='red', linestyle='dotted')
plt.legend(['Real values', 'Predicted values'])
plt.show();

As we can see from the graph, the model made accurate predictions of the Retail Index for the state of São Paulo.

### Conclusion


The conclusion that can be drawn from applying the ARIMA model to the dataset is that the final predictions were good, with a mean error of 5.91%. Some information, such as the presence of seasonality and the verification that the data needed to be differenced using the Dickey Fuller test, was necessary for the optimal fit of the model.